In [1]:
#!/usr/bin/env perl
use strict;
use warnings;
use feature qw/ say current_sub state /;
use Data::Dumper;
use lib '.';
use toolkit;
use graphql;
use adventofcode;

$Data::Dumper::Sortkeys = 1;

1

In [2]:
my $practice_input = qq/
###############
#...#...#.....#
#.#.#.#.#.###.#
#S#...#.#.#...#
#######.#.#.###
#######.#.#...#
#######.#.###.#
###..E#...#...#
###.#######.###
#...###...#...#
#.#####.#.###.#
#.#...#.#.#...#
#.#.#.#.#.#.###
#...#...#...###
###############
/;


###############
#...#...#.....#
#.#.#.#.#.###.#
#S#...#.#.#...#
#######.#.#.###
#######.#.#...#
#######.#.###.#
###..E#...#...#
###.#######.###
#...###...#...#
#.#####.#.###.#
#.#...#.#.#...#
#.#.#.#.#.#.###
#...#...#...###
###############


In [39]:
sub subp { my ($a, $b) = @_; return [ map { $a->[$_] - $b->[$_] } 0 .. $#$a ]; }
sub addp { my ($a, $b) = @_; return [ map { $a->[$_] + $b->[$_] } 0 .. $#$a ]; }
sub mulp { my ($a, $v) = @_; return [ map { $a->[$_] * $v } 0 .. $#$a ]; }
sub stringp { my $p = $_[0] // $_; return join ',', @$p }
sub rotate90p { my $p = $_[0] // $_; return [$p->[1], -$p->[0]] }
sub rotate270p { my $p = $_[0] // $_; return [-$p->[1], $p->[0]] }
sub uniqp { my %h; @h{map { join ',', @$_ } @_} = (); map [ split ',' ], keys %h }
sub in_bounds {
    my ($p, @shape) = @_;
    return $p->[0] >= 0 && $p->[1] >= 0 && $p->[0] < $shape[0] && $p->[1] < $shape[1];
}
sub adjacent_points {
    my ($p, @shape) = @_;
    return grep in_bounds($_, @shape), map addp($p, $_), [-1,0],[1,0],[0,-1],[0,1];
}
$graphql::graphql_methods{map_distance} = sub {
        my ($self, $params) = @_;
        my $arr = $self->{arr};
        # say 'wat:', Dumper $self->{arr};
        my @shape = shape($arr);

        state %cache;

        return $cache{$arr}{stringp($self->{coord})} if exists $cache{$arr};

        my @start =
            grep defined,
            map selector('where.coord')->($_),
            flatten_nd
            map_nd_indexed graphql_query(qq/{ where(value:"$params->{from_value}") { coord } }/),
            $arr;
        my %stepped;
        my @next_points = @start;
        my $dist = 0;
        while (@next_points) {
            die 'assert distance' if $dist > 25000;
            @stepped{map stringp, @next_points} = ($dist) x @next_points;
            @next_points =
                uniqp
                grep { not exists $stepped{stringp $_} }
                grep $arr->[$_->[0]][$_->[1]] ne '#',
                map adjacent_points($_, @shape),
                @next_points;
            $dist += 1;
        }
        $cache{$arr} = \%stepped;
        return $cache{$arr}{stringp $self->{coord}};
    };

CODE(0x564ef7bd6948)

Warning: Subroutine subp redefined at reply input line 1.

Subroutine addp redefined at reply input line 2.

Subroutine mulp redefined at reply input line 3.

Subroutine stringp redefined at reply input line 4.

Subroutine rotate90p redefined at reply input line 5.

Subroutine rotate270p redefined at reply input line 6.

Subroutine uniqp redefined at reply input line 7.

Subroutine in_bounds redefined at reply input line 8.

Subroutine adjacent_points redefined at reply input line 12.


In [60]:

sub process_day20_part1 {
    my ($input) = @_;

    my $map =
        map_nd sub { $_->{value} eq '#' ? '#' : $_->{map_distance} },
        map_nd_indexed graphql_query('{ value map_distance(from_value:"S") }'),
        parse_2d_map_array $input;

    my $debug_map = string_2d_numbers_array map_nd sub { $_ eq '#' ? undef : $_ }, $map;
    # say "debug: map:\n$debug_map";

    my @shape = shape($map);

        # string_2d_map_array
        sum
        map 1,
        sort { $a <=> $b }
        grep $_ >= 100,
        flatten_nd
        map_nd sub { map $_->{save}, @{$_->{shortcuts}} },
        map_nd_indexed sub {
            my ($arr, $p, $v) = @_;
            return undef if $v ne '#';
            my @adj = grep $arr->[$_->[0]][$_->[1]] ne '#', adjacent_points($p, @shape);
            return undef if @adj < 2;

            { shortcuts => [
                map {
                    my $start_cheat = $_;
                    map {
                        my $end_cheat = $_;
                        my $diff = graphql::get_value($arr, $end_cheat) - graphql::get_value($arr, $start_cheat);
                        # say "wat: $diff";
                        # return if $diff <= 2;
                        # say 'wat';

                        $diff > 2 ? { start => $start_cheat, end => $end_cheat, save => $diff - 2 } : ()
                        # return stringp($start_cheat) . ':' . stringp($end_cheat) . ' -> ' . $diff;
                    } @adj
                }
                @adj ] }
            # '*'
        },
        $map;
}
timer sub {
    say Dumper process_day20_part1($practice_input);
    # confirm process_day20_part1($practice_input), 6;
}

$VAR1 = 0;

[time] 0 seconds


0

Warning: Subroutine process_day20_part1 redefined at reply input line 2.


In [61]:
my $res;
timer sub {
    my $input = get_challenge('2024/day/20/input');
    $res = process_day20_part1($input);
    say Dumper $res;
    # confirm $res, 342;
};

$VAR1 = 1441;

[time] 1 seconds


1

In [65]:
say post_answer('2024/day/20/answer', 1, $res);

<!DOCTYPE html>
<html lang="en-us">
<head>
<meta charset="utf-8"/>
<title>Day 20 - Advent of Code 2024</title>
<link rel="stylesheet" type="text/css" href="/static/style.css?31"/>
<link rel="stylesheet alternate" type="text/css" href="/static/highcontrast.css?1" title="High Contrast"/>
<link rel="shortcut icon" href="/favicon.png"/>
<script>window.addEventListener('click', function(e,s,r){if(e.target.nodeName==='CODE'&&e.detail===3){s=window.getSelection();s.removeAllRanges();r=document.createRange();r.selectNodeContents(e.target);s.addRange(r);}});</script>
</head><!--




Oh, hello!  Funny seeing you here.

I appreciate your enthusiasm, but you aren't going to find much down here.
There certainly aren't clues to any of the puzzles.  The best surprises don't
even appear in the source until you unlock them for real.

Please be careful with automated requests; I'm not a massive company, and I can
only take so much traffic.  Please be considerate so that everyone gets to play.

If you're

1

In [96]:

sub area_points {
    my ($p, $dist, @shape) = @_;
        grep in_bounds($_, @shape),
        map addp($p, $_),
        map { my $a = $_; my $range = $dist - abs($a); map [$a, $_], grep { $a != 0 or $_ != 0} -$range .. $range } -$dist .. $dist;
    # return grep in_bounds($_, @shape), map addp($p, $_), [-1,0],[1,0],[0,-1],[0,1];
}

sub plot_nd {
    my ($points, $v, $arr) = @_;
    my %point_map = map { stringp($_) => 1 } @$points;
    return
        map_nd_indexed sub { my ($arr, $p, $prev) = @_; exists $point_map{stringp($p)} ? $v : $prev },
        $arr
}

sub manhattan_distance {
    my ($p1, $p2) = @_;
    return sum map abs($p1->[$_] - $p2->[$_]), 0 .. $#$p1;
}
say
    string_2d_map_array
    plot_nd [ area_points([7,7], 3, 10,10) ], '#',
    map_nd sub { ' ' },
    nd_from_shape(10,10);

say 'distance:', manhattan_distance([2,1], [2,3]);

          
          
          
          
       #  
      ### 
     #####
    ### ##
     #####
      ### 
distance:2


1

Warning: Subroutine area_points redefined at reply input line 2.

Subroutine plot_nd redefined at reply input line 10.

Subroutine manhattan_distance redefined at reply input line 18.


In [116]:

sub process_day20_part2 {
    my ($input, $more_than) = @_;

    my $map =
        map_nd sub { $_->{value} eq '#' ? '#' : $_->{map_distance} },
        map_nd_indexed graphql_query('{ value map_distance(from_value:"S") }'),
        parse_2d_map_array $input;

    my $debug_map = string_2d_numbers_array map_nd sub { $_ eq '#' ? undef : $_ }, $map;
    say "debug: map:\n$debug_map";

    my @shape = shape($map);

        # string_2d_map_array
        # sum
        # map 1,
        # sort
    my %time_groups =
        group
        map $_->{save},
        # sort map { stringp($_->{start}) . '->' . stringp($_->{end}) . ' : ' . $_->{save} }
        grep $_->{save} >= $more_than,
        sort { $a->{save} <=> $b->{save} }
        map { @{$_->{shortcuts}} }
        grep defined, flatten_nd
        # map_nd sub { map $_->{save}, @{$_->{shortcuts}} },
        map_nd_indexed sub {
            my ($arr, $p, $v) = @_;
            return undef if $v eq '#';
            my @adj = grep $arr->[$_->[0]][$_->[1]] ne '#', area_points($p, 20, @shape);
            # my @adj = grep $arr->[$_->[0]][$_->[1]] ne '#', adjacent_points($p, @shape);
            # return undef if @adj < 2;

            my $start_cheat = $p;
            my $start_value = $v;
            { shortcuts => [
                map {
                    my $end_cheat = $_;
                    my $distance = manhattan_distance($start_cheat, $end_cheat);
                    my $diff = graphql::get_value($arr, $end_cheat) - $start_value;
                    # say "wat: $diff";
                    # return if $diff <= 2;
                    # say 'wat';

                    $diff > $distance ? { start => $start_cheat, end => $end_cheat, save => $diff - $distance } : ()
                    # return stringp($start_cheat) . ':' . stringp($end_cheat) . ' -> ' . $diff;
                } @adj
            ] }
            # '*'
        },
        $map;

    foreach (sort map int, keys %time_groups) {
        say "debug: $_: ", scalar @{$time_groups{$_}};
    }

    return sum map scalar @$_, values %time_groups;
}
timer sub {
    say Dumper process_day20_part2($practice_input, 50);
    confirm process_day20_part2($practice_input, 50), 285;
}

debug: map:
      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |
      |     2|     3|     4|      |    10|    11|    12|      |    26|    27|    28|    29|    30|      |
      |     1|      |     5|      |     9|      |    13|      |    25|      |      |      |    31|      |
      |     0|      |     6|     7|     8|      |    14|      |    24|      |    34|    33|    32|      |
      |      |      |      |      |      |      |    15|      |    23|      |    35|      |      |      |
      |      |      |      |      |      |      |    16|      |    22|      |    36|    37|    38|      |
      |      |      |      |      |      |      |    17|      |    21|      |      |      |    39|      |
      |      |      |    82|    83|    84|      |    18|    19|    20|      |    42|    41|    40|      |
      |      |      |    81|      |      |      |      |      |      |      |    43|      |      |      |
      |    78|    79|    80|      

1

In [118]:
my $res;
timer sub {
    my $input = get_challenge('2024/day/20/input');
    $res = process_day20_part2($input, 100);
    say Dumper $res;
    # confirm $res, 342;
};

debug: map:
      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |      |


78